In [1]:
import graphlab as gl

# Load the MNIST data (from an S3 bucket)
data = gl.SFrame('http://s3.amazonaws.com/dato-datasets/mnist/sframe/train')
test_data = gl.SFrame('http://s3.amazonaws.com/dato-datasets/mnist/sframe/test')

# Random split the training-data
training_data, validation_data = data.random_split(0.8)

# Make sure all images are of the same size (Required by Neuralnets)
for sf in [training_data, validation_data, test_data]:
  sf['image'] = gl.image_analysis.resize(sf['image'], 28, 28, 1)

net = gl.deeplearning.get_builtin_neuralnet('mnist')

print "Layers of the network "
print "--------------------------------------------------------"
print net.layers

print "Parameters of the network "
print "--------------------------------------------------------"
print net.params


[INFO] This trial license of GraphLab Create is assigned to trephine1@yahoo.com and will expire on October 12, 2015. Please contact trial@dato.com for licensing options or to request a free non-commercial license for personal or academic use.

[INFO] Start server at: ipc:///tmp/graphlab_server-10050 - Server binary: /Users/mark/Galvanize/dato-env/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1442096508.log
[INFO] GraphLab Server Version: 1.5.2


PROGRESS: Downloading http://s3.amazonaws.com/dato-datasets/mnist/sframe/train/dir_archive.ini to /var/tmp/graphlab-mark/10050/000000.ini
PROGRESS: Downloading http://s3.amazonaws.com/dato-datasets/mnist/sframe/train/objects.bin to /var/tmp/graphlab-mark/10050/000001.bin
PROGRESS: Downloading http://s3.amazonaws.com/dato-datasets/mnist/sframe/train/m_4558118e.frame_idx to /var/tmp/graphlab-mark/10050/000002.frame_idx
PROGRESS: Downloading http://s3.amazonaws.com/dato-datasets/mnist/sframe/train/m_4558118e.sidx to /var/tmp/graphlab-mark/10050/000003.sidx
PROGRESS: Downloading http://s3.amazonaws.com/dato-datasets/mnist/sframe/test/dir_archive.ini to /var/tmp/graphlab-mark/10050/000004.ini
PROGRESS: Downloading http://s3.amazonaws.com/dato-datasets/mnist/sframe/test/objects.bin to /var/tmp/graphlab-mark/10050/000005.bin
PROGRESS: Downloading http://s3.amazonaws.com/dato-datasets/mnist/sframe/test/m_310c50b3.frame_idx to /var/tmp/graphlab-mark/10050/000006.frame_idx
PROGRESS: Downloading 

In [2]:
model = gl.neuralnet_classifier.create(training_data, target='label',
                                         network = net,
                                         validation_set=validation_data,
                                         metric=['accuracy', 'recall@2'],
                                         max_iterations=3)

Using network:

### network layers ###
layer[0]: ConvolutionLayer
  init_random = xavier
  padding = 1
  stride = 2
  num_channels = 32
  num_groups = 1
  kernel_size = 3
layer[1]: MaxPoolingLayer
  padding = 0
  stride = 2
  kernel_size = 3
layer[2]: FlattenLayer
layer[3]: DropoutLayer
  threshold = 0.5
layer[4]: FullConnectionLayer
  init_sigma = 0.01
  init_random = gaussian
  init_bias = 0
  num_hidden_units = 100
layer[5]: SigmoidLayer
layer[6]: FullConnectionLayer
  init_sigma = 0.01
  init_random = gaussian
  init_bias = 0
  num_hidden_units = 10
layer[7]: SoftmaxLayer
### end network layers ###

### network parameters ###
init_random = gaussian
learning_rate = 0.1
input_shape = 1,28,28
batch_size = 100
divideby = 255
l2_regularization = 0.0
metric = accuracy,recall@2
momentum = 0.9
### end network parameters ###

PROGRESS: Computing mean image...
PROGRESS: Done computing mean image.
PROGRESS: Creating neuralnet using cpu
PROGRESS: Training with batch size = 100
PROGRESS: +-----

In [3]:
predictions = model.classify(test_data)
print predictions

+--------+-------+----------------+
| row_id | class |     score      |
+--------+-------+----------------+
|   0    |   0   | 0.998517096043 |
|   1    |   0   | 0.99977439642  |
|   2    |   0   | 0.999371349812 |
|   3    |   0   | 0.994148373604 |
|   4    |   0   | 0.999643802643 |
|   5    |   0   | 0.998987734318 |
|   6    |   0   | 0.997004210949 |
|   7    |   0   | 0.999610006809 |
|   8    |   0   |  0.9966609478  |
|   9    |   0   | 0.999795615673 |
+--------+-------+----------------+
[10000 rows x 3 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.


In [4]:
pred_top2 = model.predict_topk(test_data, k=2)
print pred_top2

+--------+-------+-------------------+
| row_id | class |       score       |
+--------+-------+-------------------+
|   0    |   0   |   0.998517096043  |
|   0    |   6   | 0.000474064901937 |
|   1    |   0   |   0.99977439642   |
|   1    |   2   | 0.000115993869258 |
|   2    |   0   |   0.999371349812  |
|   2    |   2   | 0.000200508176931 |
|   3    |   0   |   0.994148373604  |
|   3    |   8   |  0.00261444156058 |
|   4    |   0   |   0.999643802643  |
|   4    |   2   | 0.000103457256046 |
+--------+-------+-------------------+
[20000 rows x 3 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.


In [5]:
result = model.evaluate(test_data)
print "Accuracy         : %s" % result['accuracy']
print "Confusion Matrix : \n%s" % result['confusion_matrix']

Accuracy         : 0.979499995708
Confusion Matrix : 
+--------------+-----------------+-------+
| target_label | predicted_label | count |
+--------------+-----------------+-------+
|      5       |        5        |  869  |
|      0       |        5        |   3   |
|      6       |        5        |   7   |
|      8       |        5        |   2   |
|      1       |        5        |   1   |
|      9       |        5        |   2   |
|      2       |        2        |  1011 |
|      7       |        2        |   11  |
|      3       |        2        |   1   |
|      4       |        2        |   1   |
+--------------+-----------------+-------+
[64 rows x 3 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.


### Comment: effing impressive. Training/optimizing an Octave-based model to do barely a hair better took me weeks.